In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm


fna_dir = '/mnt/raid7/Dachuang/Achuan/03_phage_host/db/host'
blastn_dir = '/mnt/raid7/Dachuang/Achuan/03_phage_host/03_blastn/01_blastn_virus'
output_dir = '/mnt/raid7/Dachuang/Achuan/03_phage_host/03_blastn/03_calculate_filtered'

virus_json=json.load(open(f'/mnt/raid7/Dachuang/Achuan/03_phage_host/db/Virus_seqlen.json'))

virus_host_dict = json.load(
    open(f'/mnt/raid7/Dachuang/Achuan/03_phage_host/db/gold_virus_info.json'))
virus_all_ids=list(virus_host_dict.keys())

def mkdir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)
mkdir(output_dir)

In [2]:

def calculate_prophage(filename):
    """
    calculate the evevry prophage score
    """
    from Bio import SeqIO
    from pandas.errors import EmptyDataError
    # read blastn file
    try:
        df = pd.read_table(f"{blastn_dir}/{filename}/{filename}.tab", sep="\t", header=None)
    except EmptyDataError:
        return

    # add blastn output info
    df.columns = ["query", "subject", "identity", "alignment_length", "mismatches",
                "gap_openings", "q_start", "q_end", "s_start", "s_end", "evalue", "bit_score"]
    df = df[(df["alignment_length"] > 500) & (df["identity"] > 90.0)].copy()

    # 先统计指标：alignment_length*identity
    calculate_dict = {}
    for i in range(len(df)):
        # 防止之前不完整的id在列表中
        virus_id = df.iloc[i]['subject'].split('.')[0]
        if virus_id not in virus_all_ids:
            continue
        key = df.iloc[i]['subject']
        if key not in calculate_dict:
            calculate_dict[key] = {'single': []}
        # calculate_dict[key]['single'].append(
        #     df.iloc[i]['alignment_length']*df.iloc[i]['identity']/(df.iloc[i]['mismatches']+1))
        calculate_dict[key]['single'].append(
            df.iloc[i]['alignment_length']*df.iloc[i]['identity']/100)

    # 计算prophage 长度
    # pro_fasta_dict = SeqIO.to_dict(SeqIO.parse(
    #     open(f'{prophgae_dir}/{filename}'), 'fasta'))
    # 计算最终指标prophage_score=sum(alignment_length*identity)/prophage_length

    for key in calculate_dict:
        virus_id = key
        virus_length = virus_json[virus_id]
        """coverage因为prophage length可能比virus length大，不能说明好坏,所以取消这个指标
        prophage_length = len(pro_fasta_dict[prophage_id].seq)
        coverage = prophage_length/virus_length
        calculate_dict[key]['score'] = sum(
            calculate_dict[key]['single'])/virus_length*coverage
        """
        # calculate_dict[key]['score'] = sum(
        #     calculate_dict[key]['single'])/virus_length
        """尝试多个的添加系数
        """
        duplicate_number = len(calculate_dict[key]['single'])
        calculate_dict[key]['score'] = np.max(
            calculate_dict[key]['single'])/virus_length*(duplicate_number*0.1+1)


    # calculate_dict to table
    sorted_dict = dict(sorted(calculate_dict.items(),
                    key=lambda x: x[1]['score'], reverse=True))
    with open(f'{output_dir}/{filename}.tsv', 'w') as f:
        for key in sorted_dict:
            virus_id = key
            score = sorted_dict[key]['score']
            f.write(f'{filename}\t{virus_id}\t{score}\n')


In [3]:
!rm /mnt/raid7/Dachuang/Achuan/03_phage_host/03_blastn/03_calculate_filtered/*

In [4]:
files = os.listdir(f"{blastn_dir}")
for filename in tqdm(files):
    calculate_prophage(filename)



100%|██████████| 496/496 [00:24<00:00, 20.21it/s]


## 合并所有文件


In [5]:
!mkdir -p /mnt/raid7/Dachuang/Achuan/03_phage_host/03_blastn/04_score
! cat /mnt/raid7/Dachuang/Achuan/03_phage_host/03_blastn/03_calculate_filtered/*.tsv > /mnt/raid7/Dachuang/Achuan/03_phage_host/03_blastn/04_score/all_score_filtered.tsv

## 解析all_score.tsv

- task1： 获取每个virus-host 的 prophage score，对于每个virus找把分数最高的host当做是病毒的host
- task2： 给每个virus-host 分配标签，1代表正确，0代表错误

In [6]:
score_dir='/mnt/raid7/Dachuang/Achuan/03_phage_host/03_blastn/04_score'

In [7]:
import pandas as pd
df_score=pd.read_table(
    f'{score_dir}/all_score_filtered.tsv', header=None)
df_score.columns = ['host', 'virus', 'score']

df_score


,host,virus,score
0,GCF_000005845.2,NC_049342.1,0.277898
1,GCF_000005845.2,NC_049343.1,0.216462
2,GCF_000005845.2,NC_022749.1,0.154594
3,GCF_000005845.2,NC_049953.1,0.121097
4,GCF_000005845.2,NC_049955.1,0.113821
...,...,...,...
4364,GCF_902387845.1,NC_001416.1,0.058308
4365,GCF_902387845.1,NC_049951.1,0.029198
4366,GCF_902387845.1,NC_042057.1,0.023704
4367,GCF_903886475.1,NC_023503.1,0.110665


In [8]:
import numpy as np
# task1： 获取每个virus-host 的 prophage score，对于每个virus找把分数最高的host当做是病毒的host
""" 不重复
df_only=df_score.sort_values(by='score', ascending=False).groupby(
    'virus', as_index=False).first()
df_only
"""
"""重复
"""
df_score["rank"] = df_score.groupby("virus")["score"].rank(
    method="min", ascending=False).astype(np.int64)
df_only = df_score[df_score["rank"] == 1][["virus", "host", "score"]]
df_only


,virus,host,score
96,NC_002670.1,GCF_000006865.1,1.642917
97,NC_002667.1,GCF_000006865.1,1.012755
98,NC_002668.1,GCF_000006865.1,0.874772
99,NC_002671.1,GCF_000006865.1,0.752724
100,NC_002666.1,GCF_000006865.1,0.672722
...,...,...,...
4333,NC_024365.1,GCF_900235835.1,0.164177
4358,NC_029078.1,GCF_900637845.1,0.037622
4359,NC_001331.1,GCF_902172305.2,0.689898
4367,NC_023503.1,GCF_903886475.1,0.110665


In [9]:
# task2： 给每个virus-host 分配标签，1代表正确，0代表错误
import json

host_lineage_dict = json.load(
    open(f'/mnt/raid7/Dachuang/Achuan/03_phage_host/db/gcf_dict.json'))
virus_host_dict = json.load(
    open(f'/mnt/raid7/Dachuang/Achuan/03_phage_host/db/gold_virus_info.json'))

def species_tag(df):
    host_id = df['host']
    virus_id = df['virus'].split('.')[0]
    host_taxonomy = host_lineage_dict[host_id].split(';')[6].split('__')[1]
    virus_taxonomy_list = virus_host_dict[virus_id]['split_lineage']["species"]
    if host_taxonomy in virus_taxonomy_list:
        return 1
    else:
        return 0


def genus_tag(df):
    host_id = df['host']
    virus_id = df['virus'].split('.')[0]
    host_taxonomy = host_lineage_dict[host_id].split(';')[5].split('__')[1]
    virus_taxonomy_list = virus_host_dict[virus_id]['split_lineage']["genus"]
    if host_taxonomy in virus_taxonomy_list:
        return 1
    else:
        return 0


In [10]:

# 给df_score 分配标签，1代表正确，0代表错误
df_score['species_tag'] = df_score.apply(lambda x: species_tag(x), axis=1)
df_score['genus_tag'] = df_score.apply(lambda x: genus_tag(x), axis=1)
df_score.to_csv(
    f'{score_dir}/all_score_filtered_tag.tsv', sep='\t', index=False)


In [11]:

# 给df_only 分配标签，1代表正确，0代表错误
df_only['species_tag'] = df_only.apply(lambda x: species_tag(x), axis=1)
df_only['genus_tag']=df_only.apply(lambda x:genus_tag(x),axis=1)
df_only.to_csv(
    f'{score_dir}/only_score_filtered_tag.tsv', sep='\t', index=False)


In [12]:
df_only

,virus,host,score,species_tag,genus_tag
96,NC_002670.1,GCF_000006865.1,1.642917,1,1
97,NC_002667.1,GCF_000006865.1,1.012755,1,1
98,NC_002668.1,GCF_000006865.1,0.874772,1,1
99,NC_002671.1,GCF_000006865.1,0.752724,1,1
100,NC_002666.1,GCF_000006865.1,0.672722,1,1
...,...,...,...,...,...
4333,NC_024365.1,GCF_900235835.1,0.164177,1,1
4358,NC_029078.1,GCF_900637845.1,0.037622,1,1
4359,NC_001331.1,GCF_902172305.2,0.689898,1,1
4367,NC_023503.1,GCF_903886475.1,0.110665,1,1


In [13]:
from utils.deal import *
print("all")


summary(df_score)


all


,病毒数目,宿主数目,总样本,正样本,负样本,正确率,找到正确宿主的病毒,病毒Recover-in,病毒Recover-all
species,720,241,4369,2551,1818,0.5839,623,0.8653,0.1451
genus,720,241,4369,2885,1484,0.6603,696,0.9667,0.1620


In [14]:
print("only")

summary(df_only)


only


,病毒数目,宿主数目,总样本,正样本,负样本,正确率,找到正确宿主的病毒,病毒Recover-in,病毒Recover-all
species,720,154,745,590,155,0.7919,573,0.7958,0.1334
genus,720,154,745,690,55,0.9262,672,0.9333,0.1565
